In [ ]:
df.drop("Unnamed: 0", axis=1, inplace=True)
country_counts = df["Area"].value_counts()
countries_to_drop = country_counts[country_counts < 100].index.tolist()
df = df[~df["Area"].isin(countries_to_drop)].reset_index(drop=True)
datacorr = df.copy()
categorical_columns = datacorr.select_dtypes(include=["object"]).columns.tolist()
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for column in categorical_columns:
 datacorr[column] = label_encoder.fit_transform(datacorr[column])
sns.heatmap(datacorr.corr(), annot=True, cmap="PuOr")
plt.show()